In [1]:
import os
import os.path
import numpy as np
import matplotlib.pyplot as plt
X = []    
import cv2

#menentukan direktori/folder data citra yang akan dibuka
dirname = 'hymenoptera_data/'  

#menentukan ukuran tinggi dan lebar gambar
height = 200
width = 200
dim = (width, height)

#mengumpulkan data citra yang akan dibuka dalam satu array
tampungan= [] 
for path, subdirs, files in os.walk(dirname):
    print(path)
    for name in files:
        img_path = (os.path.join(path, name))
        if img_path.endswith("jpg"):
            img = cv2.imread(img_path) #baca path data
            
            #preprocessing data / segentasi  boleh dilakukan
            convert = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            resized=cv2.resize(convert,dim, interpolation=cv2.INTER_LINEAR)
            blur = cv2.GaussianBlur(resized,(5,5),0)            
            tampungan.append(blur.flatten())
#             print(blur.shape)
X = np.array(tampungan) 
#mengetahui ukuran dari np

#labeling
y0 = np.zeros(122)
y1 = np.ones(121)
y2 = np.full((100), 2)
y = []
y = np.concatenate((y0,y1,y2), axis=0)
print(X.shape)

Matplotlib is building the font cache; this may take a moment.


hymenoptera_data/
hymenoptera_data/1
hymenoptera_data/0
hymenoptera_data/2
(343, 120000)


In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split #library untuk train test split

#melakukan splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, stratify=y) #train size adalah persentase data test yang di-split

from builtins import range
from builtins import object
num_training = X_train.shape[0] #deklarasi untuk data training diambil dari X_train
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = X_test.shape[0] #deklarasi untuk data diambil dari Y_train
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Mengubah data citra menjadi baris
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

print("X_train: "+str(X_train.shape))
print("X_test: "+str(X_test.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))
X

X_train: (274, 120000)
X_test: (69, 120000)
y_train: (274,)
y_test: (69,)


array([[ 51, 170,  94, ...,  45, 175, 106],
       [ 18, 230, 254, ...,  36, 163,  65],
       [ 30, 253,  38, ...,  26, 255, 155],
       ...,
       [ 49,  75, 173, ...,  43, 134, 131],
       [ 19, 122,  69, ...,  20,  86,  54],
       [ 10, 108,  69, ...,  27,  39,  69]], dtype=uint8)

In [3]:
from sklearn import svm

# membangun model SVM
model = svm.SVC(C=1.0, kernel="linear" ,decision_function_shape='ovr', probability=True)
weight_model = model.fit(X_train,y_train)

In [1]:
import os
import pickle

def save_model(model, filename, path=""): 
    with open(os.path.join(path, filename), 'wb') as out_name:
        pickle.dump(model, out_name, pickle.HIGHEST_PROTOCOL)

def read_model(filename, path=""):
    with open(os.path.join(path, filename), 'rb') as in_name:
        model = pickle.load(in_name)
        return model

In [5]:
save_model(weight_model, "animal.pkl", path="") #simpan model

In [2]:
model = read_model("animal.pkl", path="") #load model

In [5]:
import requests
import json

url = 'http://webapi.test/index.php'
import cv2
from skimage.transform import resize
labels = ['semut','lebah','kucing']
face_cascade = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml')
#file haar
cap = cv2.VideoCapture('cat.mov') #read mp4 file
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        muka=resize(roi_gray,(200,200,3)) #wajib sama dengan citra inputan trainer
        l=[muka.flatten()] #wajib di flatten
        #model == model weight algoritma terbaik yg diload
        id_= int(model.predict(l)[0]) #prediksi
        conf =model.predict_proba(l)  #probabilitas akurasi tiap kelas
        name = labels[id_]+' -> '+ str(conf[0][id_]) #tulisan buat hasil
        print(name)
        if conf.all() >= 0.5: #jika treshold lebih dari 50%
          
            font = cv2.QT_FONT_NORMAL
            color = (255, 0, 0)
            stroke = 2
            cv2.putText(frame, name, (x, y), font, 1, color, stroke, cv2.LINE_AA) #teks tampil
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), (2)) #kotakan deteksi
        cv2.imshow('Object Detection', roi_gray)
        # post api
        myobj = {'nama' : labels[id_],'akurasi' : str(conf[0][id_])} #datanya
        data = '?&nama='+labels[id_]+'&akurasi=' +str(conf[0][id_])
        x = requests.get(url=url+data,verify=False) #request 
        print(x.text) #hasil masih json
        y = json.loads(x.text) #convert json string
        print(y["status"]) 
        k = cv2.waitKey(20) & 0xff
        if k == ord('q'):
            break
# cap.release()
cv2.destroyAllWindows()

kucing -> 0.8555648275310739
{"status":"Berhasil","label":"kucing","akurasi":"0.8555648275310739","data":{"nama":"kucing","akurasi":"0.8555648275310739"}}
Berhasil


/home/hapra/Asdos/api/api/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapi.test'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


kucing -> 0.8555423158231481
{"status":"Berhasil","label":"kucing","akurasi":"0.8555423158231481","data":{"nama":"kucing","akurasi":"0.8555423158231481"}}
Berhasil


/home/hapra/Asdos/api/api/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapi.test'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


kucing -> 0.8555333624598829
{"status":"Berhasil","label":"kucing","akurasi":"0.8555333624598829","data":{"nama":"kucing","akurasi":"0.8555333624598829"}}
Berhasil


...

kucing -> 0.8556840187324066
{"status":"Berhasil","label":"kucing","akurasi":"0.8556840187324066","data":{"nama":"kucing","akurasi":"0.8556840187324066"}}
Berhasil


/home/hapra/Asdos/api/api/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapi.test'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


kucing -> 0.8556373535101389
{"status":"Berhasil","label":"kucing","akurasi":"0.8556373535101389","data":{"nama":"kucing","akurasi":"0.8556373535101389"}}
Berhasil


/home/hapra/Asdos/api/api/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webapi.test'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
